In [13]:
class Climate:
    def __init__(self) -> None:
        self.tem_month=0
        self.tem_day=0
        self.rain=0
        self.moist_month=0
        self.moist_day=0
        self.light_month=0
        self.diurnal=0
        pass


In [ ]:
class Plant:
    def __init__(self,x,y,r=3) -> None:
        self.x = x
        self.y = y
        self.r = r
        self.name = ""
        self.age=0
        self.t_min=0
        self.t_max=0
        self.h_min=0
        self.h_max=0
        self.l_min=0
        self.l_max=0
        self.life=0
        self.water=0
        self.water_max=0
        self.repro_period=0
        self.repro_eff=0
        self.color=""
        pass

In [8]:
class Section:
    def __init__(self,x,y) -> None:
        self.x=x
        self.y=y
        self.hum=100
        self.sil_hum=100
        self.tem=0
        self.nut=100
        self.plants=[]
        self.density=0
        
    def get_density(self):
        return self.plants.__len__() 

    def add_plant(self,plant):      
        if (plant.x >= self.x*10 and plant.x < self.x*10 + 10 and plant.y >= self.y*10 and plant.y < self.y*10 + 10):
            self.plants.append(plant)
        self.density=self.density + 1

    def add_plants(self,plants):         #筛选并将符合条件的植物放入网格
        for plant in plants:
            self.add_plant(plant)

    def len(self):
        return self.plants.__len__()

    def print(self):
        if self.len() == 0:
            print("Sec(" + str(self.x) + ","+ str(self.y) + ") is empty") 
            return
        print("In Sec(" + str(self.x) + ","+ str(self.y) + ")")
        for plant in self.plants:
            print(plant.name +" " + str(plant.x) + " " + str(plant.y) + ";")

In [10]:
import math
from typing import List
# Iterate the status of Plant

# Plant(k+1) = Plant(k) + Local->Plant + Plant->Plant

# Influence on Plants' health point

def HealthPointIteration(plant:Plant, section:Section, surrounding_Plants:List[Plant]):
    
    # Soil Nutrient
    HP = plant.life + 0.1 * section.nut
    
    # Local Temperature
    if plant.t_min > section.tem:
        HP = HP - 40 * (plant.t_min - section.tem)
    elif plant.t_max < section.tem:
        HP = HP + 40 * (plant.t_max - section.tem)

    # Water Point
    if plant.water < 0.6 * plant.water_max:
        HP = HP *(1 - 0.5 * (0.6 * plant.water_max - plant.water) / plant.water_max )

    # Plant to Plat Interaction
    for sp in surrounding_Plants:
        if sp.name == "Argy" and plant.name != "Argy":
            HP = HP - 20

    # Environmental damage
    env_dam_reduce = 0
    for sp in surrounding_Plants:
        if sp.name == "Cactus" or sp.name == "Thorn":
            env_dam_reduce = env_dam_reduce + 1
    HP = HP - min(100, 0.1 * HP * (1-(math.atan2(env_dam_reduce)*2 / math.pi)*0.5))

    return HP

In [20]:
def WaterPointIteration(plant:Plant, section:Section, surrounding_Plants:List[Plant]):
    
    # Surrounding Humidity
    WP = min(plant.water +  0.3 * section.hum, plant.water_max)

    # Local Temperature
    WP = WP - 4 * max(0, section.tem)

    #  Soil Humidity
    desired_water = plant.water_max - plant.water
    total_water_max = plant.water_max
    for sp in surrounding_Plants:
        desired_total_water = desired_total_water + sp.water_max - sp.water
        total_water_max = total_water_max + sp.water_max
    if desired_total_water < 0.4 * section.sil_hum:
        WP = plant.water_max
    else:
        WP = plat.water + (plant.water_max / total_water_max) * 0.4 * section.sil_hum

    return WP

In [15]:
# Local(k+1) = Local(k) + Climate->Local + Plant->Local

# Soil Humidity

def SoilHumidityIteration(section:Section, climate:Climate):
    
    # Climate Precipitation
    SoH = section.sil_hum + climate.rain

    # Reproduction

    # Plant Density
    SoH = SoH - section.density * 10

    # Climate Temperature
    SoH = SoH - 4 * max(0, climate.tem)

    # Plant Water Point
    desired_total_water = 0
    for sp in section.plants:
        desired_total_water = desired_total_water + sp.water_max - sp.water
    SoH = SoH - max(0.4 * section.sil_hum, desired_total_water)

    # Plant Function
    for sp in section.plants:
        if (sp.name == "Locust" or plant.name == "Poplar") and section.sil_hum < 6000:
            SoH = SoH + 0.04 * sp.water

    return SoH


In [17]:
import math

# Surrounding Humidity
def SurroundingHumidityIteration(section:Section, climate:Climate):
    
    # Climate Precipitations
    SuH = section.hum + 0.05 * climate.rain
    
    # Plant Density
    SuH = SuH * (1 - 0.5 * (2 / math.pi) * math.atan(section.density / 100000))

    # Climate Humidity
    SuH = 0.7 * SuH + 0.3 * climate.moist_day

    return SuH

In [19]:

def SoilNutrientIteration(section:Section, climate:Climate):
    # Plant Death

    # Plant Reproduction

    # Plant consumption
    SN = SN * (1- 0.6 * (2 / math.pi) * math.atan(section.density / 3000))

    #Plant Function
    for sp in section.plants:
        if sp.name == "Locust" or sp.name == "Stipa":
            SN = SN + 0.3 * sp.life

    # Daily Complement
    SN = SN + 15
    
    return SN

In [ ]:
def LocalTemperatureIteration(section:Section, climate:Climate):
    # Influence of Climate 
    rate = (2 / math.pi) * math.atan(section.density / 3000)
    AT = 0.7 * section.tem + 0.3 * (rate * climate.tem_month + (1 - rate) * climate.tem_day)

    return AT